In [1]:
import pathlib
import arviz_base as azb
import pandas as pd
import bambi as bmb

seed = 123


# Roaches 

The roaches data example comes from Chapter 8.3 of [Gelman and Hill (2007)](https://sites.stat.columbia.edu/gelman/arm/).

> The treatment and control were applied to 160 and 104 apartments, respectively, and the outcome measurement $y_i$ in each apartment $i$ was the number of roaches caught in a set of traps. Different apartments had traps for different numbers of days.

In addition to an intercept, the regression predictors for the model are the pre-treatment number of roaches `roach1`, the treatment indicator `treatment`, and a variable indicating whether the apartment is in a building restricted to elderly residents `senior`. The distribution of `roach1` is very skewed and we take a square root of it. Because the number of days for which the roach traps were used is not the same for all apartments in the sample, we include it as an exposure2 by adding $\log⁡(u_i)$ to the linear predictor $\eta_i$ and it can be specified using the offset argument in Bambi.



In [2]:
roaches = pd.read_csv("roaches.csv")
roaches["sqrt_roach1"] = roaches["roach1"].apply(lambda x: x**0.5)
roaches.head()

,y,roach1,treatment,senior,exposure2,sqrt_roach1
0,153,308.00,1,0,0.800000,17.549929
1,127,331.25,1,0,0.600000,18.200275
2,7,1.67,1,0,1.000000,1.292285
3,7,3.00,1,0,1.000000,1.732051
4,0,2.00,1,0,1.142857,1.414214


In [3]:
model_roaches_nb = bmb.Model("y ~ sqrt_roach1 + treatment + senior + offset(log(exposure2))",
                data = roaches,
                family = "negativebinomial")
idata_roaches_nb = model_roaches_nb.fit(draws=1000, tune=1000, random_seed=seed)
model_roaches_nb.predict(idata_roaches_nb, kind="response", inplace=True, random_seed=seed)

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, Intercept, sqrt_roach1, treatment, senior]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 2 seconds.


In roaches, 64% of observations are 0, and it makes sense to look how well we predict zeros or non-zeros.

In [4]:
idata_roaches_nb.posterior_predictive["y_pos"] = (idata_roaches_nb.posterior_predictive["y"] > 0).astype(int)
idata_roaches_nb.observed_data["y_pos"] = (idata_roaches_nb.observed_data["y"] > 0).astype(int)

We also add the count of roaches to constant data, so we can later retrieve this variable in examples of residual plots.

In [5]:
idata_roaches_nb["constant_data"] = azb.dict_to_dataset(
    {
        "roach count": roaches.roach1.to_numpy(),
    },
    dims={"roach count": ["__obs__"]},
    sample_dims={},
)

In [6]:
idata_roaches_nb.to_netcdf(pathlib.Path("..", "..", "data", "roaches_nb.nc"))

PosixPath('../../data/roaches_nb.nc')

Now we repeat the same steps for a zero-inflated model

In [7]:
formula = bmb.Formula(
    "y ~ sqrt_roach1 + treatment + senior + log(exposure2)",
    "psi ~ sqrt_roach1 + treatment + senior + log(exposure2)"
)

model_roaches_zinb = bmb.Model(formula,
                data = roaches,
                family = "zero_inflated_negativebinomial",
)
idata_roaches_zinb = model_roaches_zinb.fit()
model_roaches_zinb.predict(idata_roaches_zinb, kind="response", inplace=True)

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, Intercept, sqrt_roach1, treatment, senior, log(exposure2), psi_Intercept, psi_sqrt_roach1, psi_treatment, psi_senior, psi_log(exposure2)]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 8 seconds.


In roaches, 64% of observations are 0, and it makes sense to look how well we predict zeros or non-zeros.

In [8]:
idata_roaches_zinb.posterior_predictive["y_pos"] = (idata_roaches_zinb.posterior_predictive["y"] > 0).astype(int)
idata_roaches_zinb.observed_data["y_pos"] = (idata_roaches_zinb.observed_data["y"] > 0).astype(int)

We also add the count of roaches to constant data, so we can later retrieve this variable in examples of residual plots.

In [9]:
idata_roaches_zinb["constant_data"] = azb.dict_to_dataset(
    {
        "roach count": roaches.roach1.to_numpy(),
    },
    dims={"roach count": ["__obs__"]},
    sample_dims={},
)

In [10]:
idata_roaches_zinb.to_netcdf(pathlib.Path("..", "..", "data", "roaches_zinb.nc"))

PosixPath('../../data/roaches_zinb.nc')